In [10]:
import json, jsonlines
from nlgmetricverse import NLGMetricverse, load_metric
from sklearn.metrics import f1_score

ESCONV_STRATEGY = [
    "[Question]",
    "[Restatement or Paraphrasing]",
    "[Reflection of feelings]",
    "[Self-disclosure]",
    "[Affirmation and Reassurance]",
    "[Providing Suggestions]",
    "[Information]",
    "[Others]"
]

In [11]:
results = list(jsonlines.open("./GeneratedDataset/Ashokajou51_esconv-sorted-incontext-tinyllama-plm_epoch-2.json"))

In [13]:
acc = 0
error = 0

preds = []
gts = []

def get_strategy(x):
    if "]" in x:
        y = x.split("]", 1)[0] + "]"
        if y in ESCONV_STRATEGY:
            return y, ESCONV_STRATEGY.index(y)
    return "Error", -1

def clean_string(s):
    if "[Others]" in s:
        return s.split("[Others]", 1)[1]
    elif "[Question]" in s:
        return s.split("[Question]", 1)[1]
    return s.split("]", 1)[1] if "]" in s else s

for r in results:
    prediction = r["prediction"].replace("</s>", "")
    response = [r["content"].replace("|assistant|>", "") for r in results]

    # prediction_clean = clean_string(prediction)
    # response_clean = clean_string(response)

    (pred_strat, pred_strat_id), (gt_strat, gt_strat_id) = get_strategy(prediction), get_strategy(response)
    if gt_strat_id >= 0 and pred_strat == gt_strat:
        acc += 1

    preds.append(pred_strat_id)
    gts.append(gt_strat_id)

print(f"acc: {acc / len(results)}")
print(f"error: {error / len(results)}")
print("macro F1", f1_score(preds, gts, average="macro"))

predictions = [clean_string(x['prediction']).replace("</s>", "") for x in results]
references = [clean_string(x['content']) for x in results]

metrics = [
    load_metric("bleu", resulting_name="bleu_1", compute_kwargs={"max_order": 1}),
    load_metric("bleu", resulting_name="bleu_2", compute_kwargs={"max_order": 2}),
    load_metric("bleu", resulting_name="bleu_4", compute_kwargs={"max_order": 4}),
    load_metric("rouge"),
    load_metric("f1"),
]
scorer = NLGMetricverse(metrics=metrics)
scores = scorer(predictions=predictions, references=references, reduce_fn="mean")
print(json.dumps(scores, indent=4))


KeyError: 'content'

In [6]:
predictions[0]

' What would you like to talk about?'